In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
%matplotlib notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
f=np.array([0.5,0,-0.5])
b=np.array([0.9,1.1,1])
r=np.array([-0.05,0.15,0])

B=np.vstack([np.ones(3),b]).T
B

S=np.diag(np.array([0.2,0.2,0.25])**2)
S
Sinv=np.diag(1/np.diag(S))
Sinv

array([[1. , 0.9],
       [1. , 1.1],
       [1. , 1. ]])

array([[0.04  , 0.    , 0.    ],
       [0.    , 0.04  , 0.    ],
       [0.    , 0.    , 0.0625]])

array([[25.,  0.,  0.],
       [ 0., 25.,  0.],
       [ 0.,  0., 16.]])

In [20]:
# OLS
Q=B.T.dot(B)
Q
m=np.linalg.inv(Q).dot(B.T.dot(r))
m
e=r-B.dot(m)
e
e.sum()

array([[3.  , 3.  ],
       [3.  , 3.02]])

array([-0.96666667,  1.        ])

array([ 0.01666667,  0.01666667, -0.03333333])

-2.5396351688300456e-15

In [9]:
results = sm.OLS(r, B).fit()
results.summary()
results.resid

/home/alex/anaconda3/lib/python3.7/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     12.00
Date:                Sat, 04 May 2019   Prob (F-statistic):              0.179
Time:                        07:25:36   Log-Likelihood:                 6.9865
No. Observations:                   3   AIC:                            -9.973
Df Residuals:                       1   BIC:                            -11.78
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9667      0.290     -3.338      0.185      -4.647       2.714
x1             1.0000      0.289      3.464      0.179      -2.668       4.668
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.500
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.531
Skew:                          -0.707   Prob(JB):                        0.767
Kurtosis:                       1.500   Cond. No.                         24.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

array([ 0.01666667,  0.01666667, -0.03333333])

In [25]:
# weighted regression
Q=B.T.dot(Sinv).dot(B)
Q
m=np.linalg.inv(Q).dot(B.T.dot(Sinv).dot(r))
m # factor returns are different
ew=r-B.dot(m) # residual returns are different
ew
ew.sum() # and don't sum up to zero
# re-weighted residuals
reew=ew*np.diag(Sinv)/np.diag(Sinv).sum()
reew
reew.sum() # (WLS residuals * WLS weights) sum up to zero
e/ew # WLS residuals are different from OLS residualst
e/reew # re-weighted WLS residuals are proportional to OLS residuals
ew/reew

array([[66. , 66. ],
       [66. , 66.5]])

array([-0.96212121,  1.        ])

array([ 0.01212121,  0.01212121, -0.03787879])

-0.013636363636364321

array([ 0.00459137,  0.00459137, -0.00918274])

-2.3592239273284576e-16

array([1.375, 1.375, 0.88 ])

array([3.63, 3.63, 3.63])

array([2.64 , 2.64 , 4.125])

In [26]:
results = sm.WLS(r, B,weights=np.diag(Sinv)).fit()
results.summary()
results.resid

/home/alex/anaconda3/lib/python3.7/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.943
Model:                            WLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     16.50
Date:                Sat, 04 May 2019   Prob (F-statistic):              0.154
Time:                        07:51:13   Log-Likelihood:                 7.2410
No. Observations:                   3   AIC:                            -10.48
Df Residuals:                       1   BIC:                            -12.28
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9621      0.247     -3.893      0.160      -4.102       2.178
x1             1.0000      0.246      4.062      0.154      -2.128       4.128
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.485
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.531
Skew:                          -0.707   Prob(JB):                        0.767
Kurtosis:                       1.500   Cond. No.                         23.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

array([ 0.01212121,  0.01212121, -0.03787879])

In [35]:
# OLS in sklearn
reg = LinearRegression(fit_intercept=False).fit(B, r)
reg.score(B, r) # r-squared
reg.coef_ # coefficients aka factor returns
r-reg.predict(B) # residuals

0.9230769230769232

array([-0.96666667,  1.        ])

array([ 0.01666667,  0.01666667, -0.03333333])

In [36]:
# WLS in sklearn
reg = LinearRegression(fit_intercept=False).fit(B, r,np.diag(Sinv))
reg.score(B, r) # r-squared
reg.coef_ # coefficients aka factor returns
r-reg.predict(B) # residuals

0.9202161474888744

array([-0.96212121,  1.        ])

array([ 0.01212121,  0.01212121, -0.03787879])